In [ ]:
import os
import sys
from IPython.display import clear_output

modulepath = os.path.abspath(os.path.join(os.getcwd(), '../'))
sys.path.insert(0, modulepath)

from interact import System

# Working with individual structures and trajectories

All import and export of individual structures and MD trajectories is handelded by the `System` class. This is a smart wrapper around functionality of the excellent *MDTraj* library that takes care of the heavy lifting of importing and exporting many of the popular [trajectory formats](http://mdtraj.org/1.9.0/index.html).

Being a wrapper, the `System` class accepts the same arguments as *MDTraj* loaders; a trajectory or structure file and an optional topology file using the `top` keyword argument. In addition, it accepts a Tripos MOL2 file format as source of Tripos SYBYL atom types used in many of the analysis routines. The `System` class uses lazy loading for efficient out-of-core analysis of large trajectory files.

The `System` class returns `TopologyDataFrame` or `TopologySeries` objects that are extended [*Pandas*](https://pandas.pydata.org) DataFrame and Series objects. The extension towards Pandas makes all the powerfull data analysis functionality of this popular library available for structure analysis.

## 1.0 Loading single structures

A single model PDB file is analogous to a single frame trajectory. A timestep is not defined in this situation, starttime is 0 and the number of frames is 1.

In [ ]:
# Loading a single RCSB Protein Data Bank (pdb) file from disk
pdb = os.path.join(modulepath, 'tests/files/dnmt.pdb')
molsys = System(pdb)

# Display some general system information
print(molsys.timestep)
print(molsys.starttime)
print(len(molsys)) # Number of frames

# Access TopologyDataFrame. This equals the first and in this case only frame 
print(molsys.topology)

The `System.topology` object is a TopologyDataFrame object representing the topology of the system. It is created once at initialization of the molecular System class by parsing the first frame of the provided trajectory or structure file. In case of a single model PDB file the `topology` attribute and references coordinate set represent the PDB itself. The *Pandas* TopologyDataFrame object can be queried to obtain information on the structure. 

In [ ]:
# Get unique counts for every column. Equals number of atoms, residue, chains and segments among other
print(molsys.topology.nunique())

# Get residue counts for the whole system
print(molsys.topology.groupby('resName').count())

## 2.0 Loading multi-model structure files

A structure file containing multiple models (like a multi-model PDB file) is regarded as a trajectory containing
multiple frames.

In [ ]:
# Loading a multi-model RCSB Protein Data Bank (pdb) file from disk
pdb = os.path.join(modulepath, 'tests/files/2mru.pdb')
molsys = System(pdb)

# Display some general system information
print(molsys.timestep)
print(molsys.starttime)
print(len(molsys)) # Number of frames

The `System` class offers a number of methods to iterate over availbel frames or derive specific frame selections

In [ ]:
# Iterate over frames, one frame at a time
for frame, nr in molsys.iter_frames():
    print(nr)

# Or using the shortcut __iter__
for frame, nr in molsys:
    print(nr)

## 3.0 Loading molecular dynamics trajectories

In [ ]:
xtc = os.path.join(modulepath, 'tests/files/dnmt.xtc')
pdb = os.path.join(modulepath, 'tests/files/dnmt.pdb')
mol = os.path.join(modulepath, 'tests/files/dnmt.mol2')

In [ ]:
top = System(xtc, top=pdb, mol2file=mol)
print(top.timestep)
print(top.starttime)

# Using `iter_frames` for lazy loading frames
for frame, fn in top.iter_frames(start=100, stop=200, step=10):
    print(fn, frame.time)

# Shortcut using __getitem__
print(top[13])

for frame, fn in top[:20:2]:
    print(fn, frame.time, frame.coord.mean())

# NOTE: using buildin 'list' to generate a list from the generator
#       will not work as it will not copy the coordinate frame in
#       the respective frame
sel = list(top[1:4])

In [ ]:
# Example trajectory contains 250 frames

top = System(xtc, top=gro, mol2file=mol)
for frame, fn in top.iter_frames(chunk=100):
    
    #dna = frame[frame['resSeq'].isin((403, 432))]
    #dna.distances()
    
    #cf = dna[dna['resSeq'] == 403].contacts(target=dna)
    #cf = eval_hbonds(cf, dna)
    #print(cf[cf['contact'] != 'nd'])
    
    #clear_output(wait=True)
    print('Processed frame {0}'.format(fn))

print('Finished trajectory analysis')